In [1]:
import pandas
from kubeflow.metadata import metadata
from datetime import datetime
from uuid import uuid4

In [2]:
METADATA_STORE_HOST = "metadata-grpc-service.kubeflow" # default DNS of Kubeflow Metadata gRPC serivce.
METADATA_STORE_PORT = 8080

In [3]:
ws = metadata.Workspace(
   store=metadata.Store(grpc_host=METADATA_STORE_HOST, grpc_port=METADATA_STORE_PORT),
   name="example_workspace",
   description="workspace example",
   labels={"n1": "v1"})

In [4]:
r = metadata.Run(
    workspace=ws,
    name="run-" + datetime.utcnow().isoformat("T") ,
    description="a run in ws",
)

In [5]:
exec = metadata.Execution(
    name = "execution" + datetime.utcnow().isoformat("T") ,
    workspace=ws,
    run=r,
    description="execution example",
)
print("An execution was created with id %s" % exec.id)

An execution was created with id 26


In [6]:
date_set_version = "data_set_version_" + str(uuid4())
data_set = exec.log_input(
        metadata.DataSet(
            description="an example data",
            name="mytable-dump",
            owner="owner@my-company.org",
            uri="file://path/to/dataset",
            version=date_set_version,
            query="SELECT * FROM mytable"))
print("Data set id is {0.id} with version '{0.version}'".format(data_set))

Data set id is 37 with version 'data_set_version_05f96b25-0556-4ce5-8fbb-3622499579cd'


In [7]:
model_version = "model_version_" + str(uuid4())
model = exec.log_output(
    metadata.Model(
            name="MNIST",
            description="model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist",
            model_type="neural network",
            training_framework={
                "name": "tensorflow",
                "version": "v1.0"
            },
            hyperparameters={
                "learning_rate": 0.5,
                "layers": [10, 3, 1],
                "early_stop": True
            },
            version=model_version,
            labels={"mylabel": "l1"}))
print(model)
print("\nModel id is {0.id} and version is {0.version}".format(model))

kubeflow.metadata.metadata.Model(workspace=None, name='MNIST', description='model to recognize handwritten digits', owner='someone@kubeflow.org', uri='gcs://my-bucket/mnist', version='model_version_da8e3c6b-0648-4919-8e9c-c924f6ed92a8', model_type='neural network', training_framework={'name': 'tensorflow', 'version': 'v1.0'}, hyperparameters={'learning_rate': 0.5, 'layers': [10, 3, 1], 'early_stop': True}, labels={'mylabel': 'l1'}, id=38, create_time='2020-05-02T03:21:41.028499Z', kwargs={})

Model id is 38 and version is model_version_da8e3c6b-0648-4919-8e9c-c924f6ed92a8


In [8]:
metrics = exec.log_output(
    metadata.Metrics(
            name="MNIST-evaluation",
            description="validating the MNIST model to recognize handwritten digits",
            owner="someone@kubeflow.org",
            uri="gcs://my-bucket/mnist-eval.csv",
            data_set_id=str(data_set.id),
            model_id=str(model.id),
            metrics_type=metadata.Metrics.VALIDATION,
            values={"accuracy": 0.95},
            labels={"mylabel": "l1"}))
print("Metrics id is %s" % metrics.id)

Metrics id is 39


In [9]:
serving_application = metadata.Execution(
    name="serving model",
    workspace=ws,
    description="an execution to represent model serving component",
)
# Noticed we use model name, version, uri to uniquely identify existing model.
served_model = metadata.Model(
    name="MNIST",
    uri="gcs://my-bucket/mnist",
    version=model.version,
)
m=serving_application.log_input(served_model)
print("Found the mode with id {0.id} and version '{0.version}'.".format(m))

Found the mode with id 38 and version 'model_version_da8e3c6b-0648-4919-8e9c-c924f6ed92a8'.


In [10]:
pandas.DataFrame.from_dict(ws.list(metadata.Model.ARTIFACT_TYPE_NAME))

,id,workspace,run,create_time,version,owner,description,name,model_type,uri,training_framework,hyperparameters,labels,kwargs
0,23,example_workspace,run-2020-04-25T05:57:15.159394,2020-04-25T05:57:16.857712Z,model_version_9da208bf-40bf-4937-8093-09f90e88...,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}
1,38,example_workspace,run-2020-05-02T03:21:40.889874,2020-05-02T03:21:41.028499Z,model_version_da8e3c6b-0648-4919-8e9c-c924f6ed...,someone@kubeflow.org,model to recognize handwritten digits,MNIST,neural network,gcs://my-bucket/mnist,"{'name': 'tensorflow', 'version': 'v1.0'}","{'learning_rate': 0.5, 'layers': [10, 3, 1], '...",{'mylabel': 'l1'},{}


In [11]:
print("Model id is %s\n" % model.id)
    
model_events = ws.store.get_events_by_artifact_ids([model.id])

execution_ids = set(e.execution_id for e in model_events)
print("All executions related to the model are {}".format(execution_ids))
# assert execution_ids == set([serving_application.id, exec.id])

trainer_events = ws.store.get_events_by_execution_ids([exec.id])
artifact_ids = set(e.artifact_id for e in trainer_events)
print("All artifacts related to the training event are {}".format(artifact_ids))# assert artifact_ids == set([model.id, metrics.id, data_set.id])

Model id is 38

All executions related to the model are {26, 27}
All artifacts related to the training event are {37, 38, 39}
